In [2]:
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver
import pandas as pd
import datetime

In [236]:
# 사이트 로그인
driver = webdriver.Chrome('chromedriver')
driver.get("https://weibo.com/login.php")
driver.implicitly_wait(20)
# 창 최대화
driver.maximize_window()
time.sleep(1)

username = 'globalgg1@gg.go.kr'
password = 'zhsxpscm123!'

# id, passward 입력하기
element_id = driver.find_element_by_id('loginname')
element_id.send_keys(username)
time.sleep(1)
element_password = driver.find_element_by_name('password')
element_password.send_keys(password)

time.sleep(1.5)

##로그인버튼 클릭
driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()

<ipython-input-236-7107530d94fb>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element_id = driver.find_element_by_id('loginname')
<ipython-input-236-7107530d94fb>:16: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  element_password = driver.find_element_by_name('password')
<ipython-input-236-7107530d94fb>:22: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()


In [237]:
url = 'https://weibo.com/p/1001066011831795/home?from=page_100106&mod=TAB#place'
# webdriver 로드
driver.get(url)
driver.implicitly_wait(3)

# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
soup.title

<title>京畿道官博的微博_微博</title>

In [238]:
raw_contents = soup.find_all('div', class_='WB_text W_f14')
raw_contents[0].text.strip()

'京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地方。离首都首尔的市中心的光华门有40km远，因为从地图上看，它是目前分裂的韩半岛的中心。东豆川市迎来适合旅行的季节春天，有着很多历史和故事的场所。历史上具有重要价值的名胜之一"逍遥山"。...展开全文c'

In [239]:
content_list = []
for i in range(len(raw_contents)):
    content = raw_contents[i].text.strip()
    content_list.append(content)
len(content_list)

15

In [240]:
raw_number_view = soup.find_all('i', class_='S_txt2')
len(raw_number_view)

15

In [241]:
temp = raw_number_view[1].attrs['title']
temp[9:-1]

'3193'

In [242]:
temp = raw_number_view[2].attrs['title']
int(temp[9:-1])

10686

In [243]:
number_view_list = []
for i in range(len(raw_number_view)):
    temp = raw_number_view[i].attrs['title']
    number_view = int(temp[9:-1])
    number_view_list.append(number_view)
len(number_view_list), number_view_list

(15,
 [3700,
  3193,
  10686,
  5558,
  7667,
  11292,
  11280,
  12742,
  14243,
  14211,
  14551,
  18309,
  16642,
  17800,
  18773])

In [244]:
raw_info = soup.find_all('span', 'line S_line1')
temp = raw_info[3]
temp.find_all('em')[-1]

<em>1</em>

In [245]:
raw_info = soup.find_all('span', 'line S_line1')
temp = raw_info[7]
temp.find_all('em')[-1]

<em>1</em>

In [246]:
raw_info = soup.find_all('span', 'line S_line1')
temp = raw_info[11]
temp.find_all('em')[-1]

<em>3</em>

In [247]:
raw_info = soup.find_all('span', 'line S_line1')
temp = raw_info[15]
temp.find_all('em')[-1]

<em>2</em>

In [248]:
number_like_list = []
num_range = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range):
    num = ( i + 1 ) * 4 - 1
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    number_like = int(temp)
    number_like_list.append(number_like)
len(number_like_list), number_like_list

(15, [1, 1, 3, 2, 2, 2, 4, 3, 3, 3, 5, 1, 3, 2, 3])

In [249]:
raw_number_comment = soup.find_all('span', 'line S_line1')
raw_number_comment[1]

<span class="line S_line1" node-type="forward_btn_text"><span><em class="W_ficon ficon_forward S_ficon"></em><em>转发</em></span></span>

In [250]:
raw_number_comment[5]

<span class="line S_line1" node-type="forward_btn_text"><span><em class="W_ficon ficon_forward S_ficon"></em><em>转发</em></span></span>

In [251]:
raw_number_comment[2]

<span class="line S_line1" node-type="comment_btn_text"><span><em class="W_ficon ficon_repeat S_ficon"></em><em>评论</em></span></span>

In [252]:
raw_number_comment[6]

<span class="line S_line1" node-type="comment_btn_text"><span><em class="W_ficon ficon_repeat S_ficon"></em><em>评论</em></span></span>

In [253]:
number_share_list = []
num_range = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range):
    num = ( i + 1 ) * 4 - 3
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    if '转发' in temp:
        number_share_list.append(0)
    else:
        number_like = int(temp)
        number_share_list.append(number_like)
len(number_share_list), number_share_list

(15, [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1])

In [254]:
number_comment_list = []
num_range_like = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range_like):
    num = ( i + 1 ) * 4 - 2
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    if '评论' in temp:
        number_comment_list.append(0)
    else:
        number_like = int(temp)
        number_comment_list.append(number_like)
len(number_comment_list), number_comment_list

(15, [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0])

In [267]:
raw_date = soup.find_all('div', 'WB_from S_txt2')
temp = raw_date[0].find('a', 'S_txt2')
temp.attrs['title'][:10]

'2022-03-02'

In [268]:
raw_date = soup.find_all('div', 'WB_from S_txt2')
temp = raw_date[1].find('a', 'S_txt2')
temp.attrs['title'][:10]

'2022-02-28'

In [269]:
len(raw_date)

15

In [270]:
date_list = []
raw_date = soup.find_all('div', 'WB_from S_txt2')
for i in range(len(raw_date)):
    temp = raw_date[i].find('a', 'S_txt2')
    date = temp.attrs['title'][:10]
    date_list.append(date)
len(date_list), date_list

(15,
 ['2022-03-02',
  '2022-02-28',
  '2022-02-24',
  '2022-02-22',
  '2022-02-18',
  '2022-02-15',
  '2022-02-10',
  '2022-02-08',
  '2022-02-06',
  '2022-02-04',
  '2022-02-01',
  '2022-01-27',
  '2022-01-26',
  '2022-01-24',
  '2022-01-20'])

In [274]:
number_of_follower = soup.find_all('strong', 'W_f16')
number_of_follower[1].text

'530741'

In [275]:
number_follower_list = []
number_of_follower = soup.find_all('strong', 'W_f16')
num = int(number_of_follower[1].text)
number_follower_list.append(num)
for i in range(len(date_list)-1):
    number_follower_list.append(0)
len(number_follower_list), number_follower_list

(15, [530741, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [276]:
data = {'날짜':date_list,
        '제목':content_list,
        '조회수':number_view_list,
        '좋아요_수':number_like_list,
        '공유수':number_share_list,
        '댓글수':number_comment_list,
        '팔로워_수':number_follower_list
}
df = pd.DataFrame(data)
df

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수
0,2022-03-02,京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地...,3700,1,0,0,530741
1,2022-02-28,京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...,3193,1,0,0,0
2,2022-02-24,历史遗迹和现代共存的京畿道光明市，我们家族的旅游路线。京畿道光明市是由于首尔地区变大而产生的...,10686,3,0,0,0
3,2022-02-22,"世界文化遗产之都,京畿道广州市，自然与历史共存的旅行在京畿道里，广州市是面积比较大的城市。空...",5558,2,0,0,0
4,2022-02-18,京畿道北部文化与教育中心，介绍一下议政府的代表性旅游景区。议政府是位于京畿道北部，首都的关口...,7667,2,0,1,0
5,2022-02-15,以家人旅行出名的城市，京畿道杨平郡羊群牧场京畿道杨平郡是包括电影和电视剧中无数登场的著名景点...,11292,2,1,0,0
6,2022-02-10,有谁还没有去过？👀我们的孩子们更喜欢的京畿道利川市必去的旅行路线👨‍👦🌟 雪峰公园代表利川市...,11280,4,0,0,0
7,2022-02-08,介绍一下京畿道安山市家族旅行路线。大韩民国安山市作为外国人居住众多的城市，有着以首都圈内最好...,12742,3,0,1,0
8,2022-02-06,保存着历史的自豪的城市富川，大韩民国历史旅行路线京畿道富川市是具有独特个性的城市。从很早的先...,14243,3,0,0,0
9,2022-02-04,这里是被得天独厚的自然环境包围的美丽的城市，京畿道果川市。作为随处都可以感受到文化艺术的幸福...,14211,3,0,0,0


In [167]:
mod_df = df.iloc[:3,:]
mod_df

,제목,조회수,좋아요 수,공유수,댓글수
0,京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地...,3673.0,1,0,0
1,京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...,3176.0,1,0,0
2,历史遗迹和现代共存的京畿道光明市，我们家族的旅游路线。京畿道光明市是由于首尔地区变大而产生的...,11000.0,3,0,0


# weibo 자료 수집

In [219]:
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver
import pandas as pd
from datetime import datetime

In [280]:
def crawling():

    url = 'https://weibo.com/p/1001066011831795/home?from=page_100106&mod=TAB#place'
    # webdriver 로드
    driver.get(url)    
    driver.implicitly_wait(3)

    # 페이지 크롤링
    soup = bs(driver.page_source, 'lxml')
    print("타이틀 : ", soup.title.text)

    # 타이틀 수집
    raw_contents = soup.find_all('div', class_='WB_text W_f14')
    content_list = []
    num = len(raw_contents)
    for i in range(num):
        content = raw_contents[i].text.strip()
        content_list.append(content)
    print('원본 게시글 수 : ', len(raw_contents))
    print('게시글 수 : ', len(content_list))

    # 조회수 수집
    raw_number_view = soup.find_all('i', class_='S_txt2')
    number_view_list = []
    for i in range(len(raw_number_view)):
        temp = raw_number_view[i].attrs['title']
        number_view = int(temp[9:-1])
        number_view_list.append(number_view)
    print('조회수 리스트 원소수 : ', len(number_view_list))

    # 정보 수집
    raw_info = soup.find_all('span', 'line S_line1')
    # 좋아요 수 수집
    number_like_list = []
    num_range = (len(raw_info) - 3) // 4 + 1
    raw_info = soup.find_all('span', 'line S_line1')
    for i in range(num_range):
        num = ( i + 1 ) * 4 - 1
        temp = raw_info[num]
        temp = temp.find_all('em')[-1].text
        number_like = int(temp)
        number_like_list.append(number_like)
    print('좋아요 리스트 원소수 : ', len(number_like_list))

    # 공유 수 수집
    number_share_list = []
    num_range = (len(raw_info) - 3) // 4 + 1
    raw_info = soup.find_all('span', 'line S_line1')
    for i in range(num_range):
        num = ( i + 1 ) * 4 - 3
        temp = raw_info[num]
        temp = temp.find_all('em')[-1].text
        if '转发' in temp:
            number_share_list.append(0)
        else:
            number_like = int(temp)
            number_share_list.append(number_like)
    print('공유수 리스트 원소수 : ', len(number_share_list))

    # 댓글수 수집
    number_comment_list = []
    num_range_like = (len(raw_info) - 3) // 4 + 1
    raw_info = soup.find_all('span', 'line S_line1')
    for i in range(num_range_like):
        num = ( i + 1 ) * 4 - 2
        temp = raw_info[num]
        temp = temp.find_all('em')[-1].text
        if '评论' in temp:
            number_comment_list.append(0)
        else:
            number_like = int(temp)
            number_comment_list.append(number_like)
    print('댓글수 리스트 원소수 : ', len(number_comment_list))

    # 게시글 날짜 수집
    date_list = []
    raw_date = soup.find_all('div', 'WB_from S_txt2')
    for i in range(len(raw_date)):
        temp = raw_date[i].find('a', 'S_txt2')
        date = temp.attrs['title'][:10]
        date_list.append(date)
    print('날짜 리스트 원소수 : ', len(date_list))

    # 팔로워 수 수집
    number_follower_list = []
    number_of_follower = soup.find_all('strong', 'W_f16')
    num = int(number_of_follower[1].text)
    number_follower_list.append(num)
    for i in range(len(date_list)-1):
        number_follower_list.append(0)
    print('팔로워 수 리스트 원소수 : ', len(number_follower_list))

    # 데이터 프레임 생성
    df = {'날짜':date_list,
        '제목':content_list,
        '조회수':number_view_list,
        '좋아요_수':number_like_list,
        '공유수':number_share_list,
        '댓글수':number_comment_list,
        '팔로워_수':number_follower_list}
    data = pd.DataFrame(df)
    
    # 파일로 출력
    file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
    file_name = 'weibo_' + file_name + '.csv'
    path = './crawling_data/'
    data.to_csv(path+file_name, index=False, encoding='utf-8')

    return data

In [281]:
# 사이트 로그인
driver = webdriver.Chrome('chromedriver99')
driver.get("https://weibo.com/login.php")
time.sleep(5)
driver.implicitly_wait(15)

# 창 최대화
driver.maximize_window()
time.sleep(1)

username = 'globalgg1@gg.go.kr'
password = 'zhsxpscm123!'

element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
element_id.send_keys(username)
time.sleep(1)
element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')
element_password.send_keys(password)

time.sleep(1.5)

##로그인버튼 클릭
driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()

<ipython-input-281-0bcb0b533b98>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver99')
<ipython-input-281-0bcb0b533b98>:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
<ipython-input-281-0bcb0b533b98>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')
<ipython-input-281-0bcb0b533b98>:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()


In [282]:
df = crawling()

타이틀 :  京畿道官博的微博_微博
원본 게시글 수 :  0
게시글 수 :  0
조회수 리스트 원소수 :  0
좋아요 리스트 원소수 :  0
공유수 리스트 원소수 :  0
댓글수 리스트 원소수 :  0
날짜 리스트 원소수 :  0
팔로워 수 리스트 원소수 :  1


ValueError: arrays must all be same length

In [228]:
df

,제목,조회수,좋아요 수,공유수,댓글수
0,京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地...,3699,1,0,0
1,京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...,3192,1,0,0
2,历史遗迹和现代共存的京畿道光明市，我们家族的旅游路线。京畿道光明市是由于首尔地区变大而产生的...,10685,3,0,0
3,"世界文化遗产之都,京畿道广州市，自然与历史共存的旅行在京畿道里，广州市是面积比较大的城市。空...",5557,2,0,0
4,京畿道北部文化与教育中心，介绍一下议政府的代表性旅游景区。议政府是位于京畿道北部，首都的关口...,7666,2,0,1
5,以家人旅行出名的城市，京畿道杨平郡羊群牧场京畿道杨平郡是包括电影和电视剧中无数登场的著名景点...,11291,2,1,0
6,有谁还没有去过？👀我们的孩子们更喜欢的京畿道利川市必去的旅行路线👨‍👦🌟 雪峰公园代表利川市...,11279,4,0,0
7,介绍一下京畿道安山市家族旅行路线。大韩民国安山市作为外国人居住众多的城市，有着以首都圈内最好...,12741,3,0,1
8,保存着历史的自豪的城市富川，大韩民国历史旅行路线京畿道富川市是具有独特个性的城市。从很早的先...,14242,3,0,0
9,这里是被得天独厚的自然环境包围的美丽的城市，京畿道果川市。作为随处都可以感受到文化艺术的幸福...,14210,3,0,0
